In [1]:
from arcgis.gis import GIS
from arcgis import features
from getpass import getpass #to accept passwords in an interactive fashion
from arcgis.features import FeatureLayer
from arcgis.features import SpatialDataFrame
from copy import deepcopy
import pandas as pd
import pickle
pd.set_option('display.max_columns', None)

## Configure GIS connection

In [2]:
gis = GIS(url="http://lahub.maps.arcgis.com/home/organization.html",
          username="vivian.peng_lahub",password="") #add your ArcGIS username and password here

## Update Test Positivity Layer

#### Load loayers from ArcGIS

In [3]:
# load test positivity layer
tests_layer = gis.content.search('title:Join_Test_Positivity_County_Zip')[1].layers[0] 
update_features = tests_layer.query().features

#### Read in updated data

In [4]:
# read in updated data
test_positivity = pd.read_csv("data/test_positivity_zip.csv")
test_positivity["Zip"] = test_positivity["Zip"].astype(str)

In [5]:
test_positivity.head()

,Zip,Indeterminate,Need New Sample,Negative,Positive,Total,Test Posivitiy,Date
0,0,0,5,56,8,64,0.125000,2021-01-01
1,1,0,1,18,1,19,0.052632,2021-01-01
2,10001,0,0,1,0,1,0.000000,2021-01-01
3,10002,0,0,5,1,6,0.166667,2021-01-01
4,10003,0,0,1,0,1,0.000000,2021-01-01


In [7]:
# Read in list of zip codes
with open ("data/LA_county_zips.data", "rb") as filehandle:
    LA_county_zips = pickle.load(filehandle)

In [8]:
# filter for LA County
test_positivity = test_positivity[test_positivity["Zip"].isin(LA_county_zips)]

#### Update features

In [9]:
#loop through ZIPs and set new values
for v in test_positivity.values:
    [f for f in update_features if f.attributes['GEOID10']==v[0]][0].attributes["Indeterminate"]= v[1]
    [f for f in update_features if f.attributes['GEOID10']==v[0]][0].attributes["Need New Sample"]= v[2]
    [f for f in update_features if f.attributes['GEOID10']==v[0]][0].attributes['Negative'] = v[3]
    [f for f in update_features if f.attributes['GEOID10']==v[0]][0].attributes['Positive'] = v[4]
    [f for f in update_features if f.attributes['GEOID10']==v[0]][0].attributes['Total'] = v[5]
    [f for f in update_features if f.attributes['GEOID10']==v[0]][0].attributes['Test Posivitiy'] = v[6]
    [f for f in update_features if f.attributes['GEOID10']==v[0]][0].attributes['Date'] = v[7]

In [10]:
#edit features to match the updated features
for f in update_features:
    tests_layer.edit_features(updates=[f])

## Update Case/Deaths Layers

In [11]:
# read in updated data
county = pd.read_csv("data/testing_neighborhoods.csv")
county = county[county["Neighborhood"]!='MISCELLANEOUS**']

In [12]:
county.head(2)

,Neighborhood,New Cases 12.26 to 1.1,Two Weeks Cases,New Deaths 12.26 to 1.1,Case Rate 12.18 to 1.1,Adjusted Case Rate 12.18 to 1.1,population,Cumulative Confirmed Case Count 1.1,Cumulative Death Count 1.1,Date,Display Name,top_case_rate,top_new_cases
0,ADAMS-NORMANDIE,96.0,184.0,2.0,2191.587133,1895.0,8202.0,778.0,7.0,2021-01-01,Adams-Normandie,38.0,88.0
1,ALSACE,120.0,245.0,0.0,1875.539512,1670.0,12445.0,1033.0,4.0,2021-01-01,Alsace,56.0,77.0


In [13]:
def update_cases_deaths_layer(title, county):
    # load layers
    layer = gis.content.search(title)[0].layers[0] 
    update_features = layer.query().features
    
    #loop through and set new values
    for v in county.values:
        [f for f in update_features if f.attributes['COMTY_NAME']==v[0]][0].attributes["New_Cases"]= v[1]
        [f for f in update_features if f.attributes['COMTY_NAME']==v[0]][0].attributes["New_Deaths"]= v[3]
        [f for f in update_features if f.attributes['COMTY_NAME']==v[0]][0].attributes['Two_Week_Crude_IR__per_100k_'] = v[4]
        [f for f in update_features if f.attributes['COMTY_NAME']==v[0]][0].attributes['adj_case_rate'] = v[5]
        [f for f in update_features if f.attributes['COMTY_NAME']==v[0]][0].attributes['Cumulative_Confirmed_Case_Count'] = v[7]
        [f for f in update_features if f.attributes['COMTY_NAME']==v[0]][0].attributes['Cumulative_Death_Count'] = v[8]
        [f for f in update_features if f.attributes['COMTY_NAME']==v[0]][0].attributes['Date_1'] = v[9]
        [f for f in update_features if f.attributes['COMTY_NAME']==v[0]][0].attributes['top_trends_cases'] = v[11]
        [f for f in update_features if f.attributes['COMTY_NAME']==v[0]][0].attributes['top_new_cases'] = v[12]
        
    #edit features to match the updated features
    for f in update_features:
        layer.edit_features(updates=[f])

In [14]:
# layers_to_update =["title:Two_Week_Case_Rates", "title:New_Cases_in_Last_7_Days", "title:New_Deaths_in_Last_7_Days","title:Top_Trends_Case_Rate",  "title:Top_Trends_New_Cases",]
# for layer in layers_to_update:
   # update_cases_deaths_layer(layer, county)

In [15]:
update_cases_deaths_layer('title:Two_Week_Case_Rates', county)

## Update Testing Sites Layer

In [13]:
from arcgis.features import FeatureLayerCollection

In [14]:
flayer = gis.content.get("1624e8fb02e54d138e2b5206dac23998")

In [15]:
flayer_collection = FeatureLayerCollection.fromitem(flayer)

In [16]:
flayer_collection.manager.overwrite("data/mobile_testing_sites.csv")

{'success': True}

## Creating new fields

In [5]:
layer = gis.content.search('title:Two_Week_Case_Rates')[0].layers[0] 
layer_fields = layer.manager.properties.fields

In [6]:
new_fields = ['adj_case_rate']

In [8]:
template_field

{'name': 'Two_Week_Crude_IR__per_100k_',
 'type': 'esriFieldTypeDouble',
 'actualType': 'decimal',
 'alias': 'Two Week Crude IR* (per 100k)',
 'sqlType': 'sqlTypeDecimal',
 'nullable': True,
 'editable': True,
 'visible': True,
 'domain': None,
 'defaultValue': None}

In [7]:
template_field = dict(deepcopy(layer_fields[5]))

In [9]:
#set up new definition and add to dictionary
fields_to_be_added = []

#create ammended dictionary for each new field
for new_field_name in new_fields:
    current_field = deepcopy(template_field)
    current_field['name'] = new_field_name.lower()
    current_field['alias'] = new_field_name
    fields_to_be_added.append(current_field)
    
    
#add definition
layer.manager.add_to_definition({'fields':fields_to_be_added})

{'success': True}